In [3]:
# Simple CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

class SimpleCNN:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize model with input shape
        model = Sequential()
        input_shape = (height, width, depth)
        chan_dim = -1
        
        # first CONV=>RELU=>BN layer set
        model.add(Conv2D(32, (3,3), strides=(2,2), padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        
        # Second CONV=>RELU=>BN layer set
        model.add(Conv2D(64, (3,3), strides=(2,2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        
        # First and only set of FC=>RELU layers
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation('softmax'))
        
        return model

ERROR! Session/line number was not unique in database. History logging moved to new session 1230


In [4]:
# Generate Adversarial image
from tensorflow.keras.losses import MSE
import tensorflow as tf

def generate_image_adversary(model, image, label, eps=2/255.0):
    # cast the image
    image = tf.cast(image, tf.float32)
    
    # record gradients
    with tf.Gradient.Tape() as tape:
        # explicitly indicate that our image should be tracked for growing gradients
        tape.watch(image)
        
        # use model to make predictions on input image and compute loss
        pred = model(image)
        loss = MSE(label, pred)
        
    # calculate gradients of loss w.r.t image then compute sign of gradient
    gradient = tape.gradient(loss, image)
    signed_grad = tf.sign(gradient)
    
    # construct the image adversary
    adversary = (image + (signed_grad * eps)).numpy()
    
    return adversary

In [5]:
# training
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2 as cv

# load mnist dataset and scale the pixel values to range [0, 1]
print('[INFO] loading MNIST dataset...')
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0

# add a chennel dimension to images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

# one hot encode labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# initialize optimizer and model
print('[INFO] Compiling model...')
opt = Adam(lr=1e-3)
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# train simple CNN on MNIST
print('[INFO] Training Model...')
model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, 
          epochs=10, verbose=1)

# make predictions on testing set for model trained on non-adversarial images
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print(f'[INFO] loss: {loss:.4f}, accuracy: {acc:.4f}')

[INFO] loading MNIST dataset...
[INFO] Compiling model...
[INFO] Training Model...
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 19s 317us/sample - loss: 0.1982 - accuracy: 0.9407 - val_loss: 0.0572 - val_accuracy: 0.9823
Epoch 2/10
60000/60000 [==============================] - 10s 171us/sample - loss: 0.0757 - accuracy: 0.9770 - val_loss: 0.0513 - val_accuracy: 0.9841
Epoch 3/10
60000/60000 [==============================] - 10s 172us/sample - loss: 0.0572 - accuracy: 0.9821 - val_loss: 0.0460 - val_accuracy: 0.9854
Epoch 4/10
60000/60000 [==============================] - 10s 170us/sample - loss: 0.0475 - accuracy: 0.9851 - val_loss: 0.0535 - val_accuracy: 0.9826
Epoch 5/10
60000/60000 [==============================] - 10s 170us/sample - loss: 0.0396 - accuracy: 0.9873 - val_loss: 0.0466 - val_accuracy: 0.9871
Epoch 6/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.0342 - accuracy: 0.9895 - v